# Fine Tuning YOLOv5 for Number Plate Detection

- Format CSV to YOLOv5 format (xcenter,ycenter,width,height)

- Save each bounding box data as a textfile with imageID under ./data/detection/labels

- store Images under ./data/data_detection/images

- Change COCO.yaml in ./yolov5/data to parse the 900 input images as training

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/detection_train.csv")

In [3]:
df.head()

,img_id,ymin,xmin,ymax,xmax
0,1.jpg,276,94,326,169
1,10.jpg,311,395,344,444
2,100.jpg,406,263,450,434
3,101.jpg,283,363,315,494
4,102.jpg,139,42,280,222


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img_id  900 non-null    object
 1   ymin    900 non-null    int64 
 2   xmin    900 non-null    int64 
 3   ymax    900 non-null    int64 
 4   xmax    900 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 35.3+ KB


### YOLOv5 Requires Bouinding Boxes to be scaled to [0,1] relative to the image, hence we will open each image and append its width and height to the dataframe for further processing

In [5]:
import os
from PIL import Image

In [7]:
def image_input(path):
    return Image.open(path)

In [20]:
def parse(df,path):
    new_rows = []
    for _,row in df.iterrows():
        filename = row['img_id']
        image = image_input(os.path.join(path,filename))
        xmin = row['xmin']
        ymin = row['ymin']
        ymax = row['ymax']        
        xmax = row['xmax']
        width,height = image.size
        
        new_row = row.copy()
        new_row['xCenter'] = (xmin+xmax)/(2*width)
        new_row['yCenter'] = (ymin+ymax)/(2*height)
        new_row['bboxWidth'] = (xmax-xmin)/width
        new_row['bboxHeight'] = (ymax-ymin)/height
        new_row['txtFileName'] = row['img_id'][:-4]

        new_rows.append(new_row)    
    
    return pd.DataFrame(new_rows)


dff = parse(df,"data/data_detection/images")

In [21]:
dff.head()

,img_id,ymin,xmin,ymax,xmax,xCenter,yCenter,bboxWidth,bboxHeight,txtFileName
0,1.jpg,276,94,326,169,0.154706,0.631027,0.088235,0.104822,1
1,10.jpg,311,395,344,444,0.656495,0.683716,0.076682,0.068894,10
2,100.jpg,406,263,450,434,0.478052,0.782450,0.234568,0.080439,100
3,101.jpg,283,363,315,494,0.504118,0.626834,0.154118,0.067086,101
4,102.jpg,139,42,280,222,0.322738,0.382998,0.440098,0.257770,102


In [ ]:
writePath  = "data/data_detection/labels"

In [26]:
for _,row in dff.iterrows():
    path = f"{writePath}/{row['txtFileName']}.txt"
    with open(path,'w') as file:
        file.write(f"0 {row['xCenter']} {row['yCenter']} {row['bboxWidth']} {row['bboxHeight']} ")

## Fine-tuning

Now that the data preprocesing is over, we will move on to fine tuning a YOLOv5 Model on our number plate dataset for bounding box detection. 

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO("yolov8n.pt")

In [4]:
model.train(data='bb_train.yaml', epochs=50, imgsz=640,val=False,device=0)

New https://pypi.org/project/ultralytics/8.3.41 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=bb_train.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning D:\dev\soulPage-task\data\detection\labels.cache... 900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 900/900 [00:00<?, ?it/s]
val: Scanning D:\dev\soulPage-task\data\detection\labels.cache... 900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 900/900 [00:00<?, ?it/s]


Plotting labels to d:\dev\soulPage-task\runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to d:\dev\soulPage-task\runs\detect\train8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.19G       1.23      2.302      1.113          7        640: 100%|██████████| 57/57 [00:13<00:00,  4.38it/s]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.19G     0.6508     0.3324     0.8583          4        640: 100%|██████████| 57/57 [00:11<00:00,  5.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:09<00:00,  2.95it/s]


                   all        900        900          1          1      0.995      0.863

50 epochs completed in 0.172 hours.
Optimizer stripped from d:\dev\soulPage-task\runs\detect\train8\weights\last.pt, 6.2MB
Optimizer stripped from d:\dev\soulPage-task\runs\detect\train8\weights\best.pt, 6.2MB

Validating d:\dev\soulPage-task\runs\detect\train8\weights\best.pt...
Ultralytics 8.3.40  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:08<00:00,  3.53it/s]


                   all        900        900          1          1      0.995      0.864
Speed: 0.3ms preprocess, 2.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to d:\dev\soulPage-task\runs\detect\train8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002196C5D0170>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [5]:
model.save("weights/bb_weights.pt")


In [10]:
results = model.predict(source='data/test/901.jpg', imgsz=640)
results[0].show()  # Display predictions


image 1/1 d:\dev\soulPage-task\data\test\901.jpg: 640x480 1 0, 36.7ms
Speed: 6.1ms preprocess, 36.7ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 480)
